http://www.bigdataschool.ru

<img src="logo.png" align="right">

# Знакомство с Trino

<hr style="border:2px solid #294A70"> </hr>

<h3 style="color: #294A70">Практические материалы</h3>

<div style="color:red; text-align: right"> Версия 1.02 от 10 января 2025 </div>

In [3]:
from trsql_h import _psql, _sql, _setSchema, _getSchema

## Утилитные функции

Функции `_sql()` и `_psql()` сделаны для работы с Trino, обе функции ожидают в качестве параметра получить запрос (в виде строки)

* _psql() выполнит запрос и выведет в output область ячейки его результат в формате ALIGNED
* _sql() выполнит запрос и возвратит результат в виде pandas dataframe 

Например, в таблице `nodes` схемы `runtime` каталога `system` можно увидеть информацию о состоянии кластера.

Функция `_sql()` вернет результат в виде pandas dataframe, которые Jupyter отрисует в виде таблицы

* с номерами строк (самая левая колонка)
* именами колонок (жирным в первой строке)
* "чересполосицей"

In [4]:
_sql("SELECT * FROM system.runtime.nodes")

,node_id,http_uri,node_version,coordinator,state
0,master,http://10.150.2.5:8080,468,True,active
1,worker-1,http://10.150.2.6:8080,468,False,active
2,worker-2,http://10.150.2.7:8080,468,False,active


Этот же запрос, выполненный функцией `_psql()` выведет в output область ячейки

* заголовок (первая строка) - имена колонок, разделенные символом `|`
* разделитель (вторая строка)
* строки результата (с разделителями `|`)
* общее количество строк в результате

In [5]:
_psql("SELECT * FROM system.runtime.nodes")

Не стоит использовать функцию `_sql()` для запросов, не возвращающих резалтсет, для этого лучше использовать `_psql()`

Функция `_sql()` может быть использована для программной обработки результата запроса, например так:

In [ ]:
resultSet = _sql("SELECT * FROM system.runtime.nodes").values.tolist() # список строк (список tuple)
for row in resultSet:
    print(f"NODE_ID: {row[0]}, STATE: {row[4]}")

### LIMIT

Некоторые запросы могут вернуть большое количество строк - не забываем использовать LIMIT в select

In [6]:
_psql("SELECT * FROM system.runtime.nodes limit 10")

Функция `_sql()` в качестве второго параметра позволяет указать, сколько строк резалтсета необходимо вернуть в результирующем датафрейме (по умолчанию - 10). Если необходимо, чтобы датафрейм содержал больше 10 строк - укажите нужное число в качестве второго параметра (при этом Trino вернет весь резалтсет, второй параметр влияет только на то, сколько строк будет вставлено в pandas dataframe).

## Втроенный connector tpcds

Trino содержит несколько "встроенных" источников данных (генераторов).

Один из них способен генерировать данные бенчмарка TPC-DS нужного объема - для этого используются соответствующие схемы.

In [7]:
_psql("show schemas in tpcds")

Схемы именуются `sfN`, где N - так называемый scale factor

* 1 = 1GB
* 10 = 10GB
* 100 = 100GB
* и т.д.

Все данные схемы должны "влезть" в NGB, например, все таблицы схемы `sf1` должны суммарно на диске занимать не больше 1ГБ (для сериализации используется CSV с разделителем запятая).

In [11]:
_sql("show tables in tpcds.sf1",100)

,Table
0,call_center
1,catalog_page
2,catalog_returns
3,catalog_sales
4,customer
5,customer_address
6,customer_demographics
7,date_dim
8,dbgen_version
9,household_demographics


Пример данных - размерность товары (`item`)

In [9]:
import pandas as pd
pd.options.display.max_columns = None # чтобы не были "пропущены" средние поля для широких таблиц

In [10]:
_sql("select * from tpcds.sf1.item limit 5")

,i_item_sk,i_item_id,i_rec_start_date,i_rec_end_date,i_item_desc,i_current_price,i_wholesale_cost,i_brand_id,i_brand,i_class_id,i_class,i_category_id,i_category,i_manufact_id,i_manufact,i_size,i_formulation,i_color,i_units,i_container,i_manager_id,i_product_name
0,1,AAAAAAAABAAAAAAA,1997-10-27,NaN,Powers will not get influences. Electoral port...,27.02,23.23,5003002,exportischolar #2 ...,3,pop ...,5,Music ...,52,ableanti ...,N/A,3663peru009490160959,spring,Tsp,Unknown,6,ought ...
1,2,AAAAAAAACAAAAAAA,1997-10-27,2000-10-26,False opportunities would run alone with a vie...,1.12,0.38,1001001,amalgamalg #1 ...,1,dresses ...,1,Women ...,294,esen stable ...,petite,516steel060826230906,rosy,Bunch,Unknown,98,able ...
2,3,AAAAAAAACAAAAAAA,2000-10-27,NaN,False opportunities would run alone with a vie...,7.11,0.38,1001001,brandbrand #4 ...,7,decor ...,7,Home ...,294,esen stable ...,N/A,516steel060826230906,sienna,Cup,Unknown,18,pri ...
3,4,AAAAAAAAEAAAAAAA,1997-10-27,1999-10-27,Normal systems would join simply different the...,1.35,0.85,3002001,importoexporti #1 ...,2,infants ...,3,Children ...,479,n stationese ...,extra large,610157moccasin018327,red,Tbl,Unknown,26,ese ...
4,5,AAAAAAAAEAAAAAAA,1999-10-28,2001-10-26,Normal systems would join simply different the...,4.00,1.76,2002002,importoimporto #2 ...,2,shirts ...,2,Men ...,220,barableable ...,petite,42214rosy28066558020,pink,Cup,Unknown,27,anti ...


Пример данных - таблица "чеков" (`store_sales`)

In [12]:
_sql("select * from tpcds.sf1.store_sales limit 5")

,ss_sold_date_sk,ss_sold_time_sk,ss_item_sk,ss_customer_sk,ss_cdemo_sk,ss_hdemo_sk,ss_addr_sk,ss_store_sk,ss_promo_sk,ss_ticket_number,ss_quantity,ss_wholesale_cost,ss_list_price,ss_sales_price,ss_ext_discount_amt,ss_ext_sales_price,ss_ext_wholesale_cost,ss_ext_list_price,ss_ext_tax,ss_coupon_amt,ss_net_paid,ss_net_paid_inc_tax,ss_net_profit
0,2451813,65495,3617,67006,591617,3428,24839,10,161,1,79,11.41,18.71,2.80,99.54,221.20,901.39,1478.09,6.08,99.54,121.66,127.74,-779.73
1,2451813,65495,13283,67006,591617,3428,24839,10,154,1,37,63.63,101.17,41.47,46.03,1534.39,2354.31,3743.29,59.53,46.03,1488.36,1547.89,-865.95
2,2451813,65495,13631,67006,591617,3428,24839,10,172,1,99,80.52,137.68,83.98,0.00,8314.02,7971.48,13630.32,0.00,0.00,8314.02,8314.02,342.54
3,2451813,65495,5981,67006,591617,3428,24839,10,280,1,14,57.37,76.30,6.10,0.00,85.40,803.18,1068.20,0.00,0.00,85.40,85.40,-717.78
4,2451813,65495,4553,67006,591617,3428,24839,10,236,1,100,25.08,36.86,0.73,0.00,73.00,2508.00,3686.00,6.57,0.00,73.00,79.57,-2435.00


Какие самые продаваемые товары? (данные таблиц бенчмарка не имеют "смысла", значения - случайны и распределены равномерно)

In [13]:
_sql("""
select 
    i_item_id,
    i_item_desc,
    sum(ss_quantity) as totquan
from
    tpcds.sf1.store_sales join tpcds.sf1.item on i_item_sk=ss_item_sk
group by 1,2
order by 3 desc
limit 10
""")

,i_item_id,i_item_desc,totquan
0,AAAAAAAAOPGDAAAA,Households help minutes. C,19617
1,AAAAAAAANGECAAAA,Well large movements shall show in the pressur...,19072
2,AAAAAAAAEJBDAAAA,"New, successful sc",19050
3,AAAAAAAAEBBCAAAA,"Nowadays single parties look variable, common ...",18674
4,AAAAAAAACDGDAAAA,Often past claims hold a little tools; effort,18658
5,AAAAAAAAAJGCAAAA,Free feet get generally lacking countries. The...,18649
6,AAAAAAAACONDAAAA,"Real, necessary names display fully. Words mus...",18594
7,AAAAAAAAKLCDAAAA,Upstream just women give with a clubs. High ad...,18583
8,AAAAAAAAOCCCAAAA,Available yards should not guess justifiably i...,18577
9,AAAAAAAAMBJBAAAA,"Other, good posts could lure. Beautiful, dead ...",18542


## Временные таблицы (connector memory)

Этот каталог удобен тем, что его не нужно "чистить" - он очищается при каждом старте сервера

Суммарный объем данных ограничен 128МБ...

Создадим схему в каталоге memory

In [14]:
_psql("create schema memory.tpcds1g")

Feb 13, 2025 2:47:20 PM org.jline.utils.Log logr
Query 20250213_144720_00078_9h98k failed: line 1:1: Schema 'memory.tpcds1g' already exists
LINE 1: create schema memory.tpcds1g
        ^




Запишем результаты предыдущего запроса в таблицу

In [15]:
_psql("""
create table memory.tpcds1g.top10items 
as
select 
    i_item_id,
    i_item_desc,
    sum(ss_quantity) as totquan
from
    tpcds.sf1.store_sales join tpcds.sf1.item on i_item_sk=ss_item_sk
group by 1,2
order by 3 desc
limit 10
""")

Feb 13, 2025 2:47:34 PM org.jline.utils.Log logr
Query 20250213_144734_00079_9h98k failed: line 1:1: Destination table 'memory.tpcds1g.top10items' already exists
LINE 1: create table memory.tpcds1g.top10items 
        ^




In [16]:
_psql("show tables in memory.tpcds1g")

In [17]:
_sql("select * from memory.tpcds1g.top10items",100)

,i_item_id,i_item_desc,totquan
0,AAAAAAAAOPGDAAAA,Households help minutes. C,19617
1,AAAAAAAANGECAAAA,Well large movements shall show in the pressur...,19072
2,AAAAAAAAEJBDAAAA,"New, successful sc",19050
3,AAAAAAAAEBBCAAAA,"Nowadays single parties look variable, common ...",18674
4,AAAAAAAACDGDAAAA,Often past claims hold a little tools; effort,18658
5,AAAAAAAAAJGCAAAA,Free feet get generally lacking countries. The...,18649
6,AAAAAAAACONDAAAA,"Real, necessary names display fully. Words mus...",18594
7,AAAAAAAAKLCDAAAA,Upstream just women give with a clubs. High ad...,18583
8,AAAAAAAAOCCCAAAA,Available yards should not guess justifiably i...,18577
9,AAAAAAAAMBJBAAAA,"Other, good posts could lure. Beautiful, dead ...",18542


In [ ]:
_psql("select count(*) from memory.tpcds1g.top10items")

<hr style="border:2px solid #294A70"> </hr>

<div style="text-align: center"> © ООО «Учебный центр «Коммерсант», 2024 </div>

<div style="text-align: center"> info@bigdataschool.ru, +7(495) 41-41-121 </div>